In [18]:
import numpy as np
from ipywidgets import HBox, VBox, Label, Layout, Button, Output
from IPython.display import display
from functools import partial
import random
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import math
from enum import Enum
from abc import ABC, abstractmethod
from dataclasses import dataclass

In [11]:
from enum import Enum
class Mark(Enum):
    NO = 0
    X = 1
    O = 2

In [12]:
class View(ABC):
    @abstractmethod
    def __init__(self):
        pass
    
    @abstractmethod
    def update(self):
        pass

In [67]:
class JupyterView(View):
    def __init__(self, board=None, turn_callback=None):
        self.board = board
        self.turn_callback = turn_callback
        self.out = Output()
        self.buttons = []
        self.next_player = Mark.NO
        self.end_game = False
        self.winner = Mark.NO
        self.draw = False
    
    def on_click(self, row, column, button):
        self.disable_all_buttons()
        self.turn_callback(row, column)
    
    def update(self):
        label = Label()
        if self.winner == Mark.NO:
            label.value = self.next_player.name + "'s turn"
        elif self.draw:
            label.value = "Draw!"
        else:
            label.value = self.winner.name + ' wins!'
        
        h, w = self.board.shape
        self.buttons = []
        buttons = []
        
        for r in range(h):
            line = []
            for c in range(w):
                button = Button(
                    description='',
                    layout=Layout(width='20px', height='20px', padding='0px')
                )
                
                button.on_click(partial(self.on_click, r, c))

                if self.board[r][c] == Mark.X.value:
                    button.description = Mark.X.name
                    button.disabled = True
                elif self.board[r][c] == Mark.O.value:
                    button.description = Mark.O.name
                    button.disabled = True
                elif self.winner != Mark.NO:
                    button.disabled = True
                
                line.append(button)
                self.buttons.append(button)
                
            buttons.append(HBox(line))
            
        self.out.clear_output()
        with self.out:
            display(VBox([label, VBox(buttons)]))
        
    def disable_all_buttons(self):
        for b in self.buttons:
            b.disabled = True
    


In [68]:
class Model(ABC):
    @abstractmethod
    def __init__(self):
        pass
    
    @abstractmethod
    def get_action(self):
        pass

In [69]:
class JupyterHumanModel(Model):
    def __init__(self):
        pass
    
    def get_action(self):
        return None

In [70]:
@dataclass
class Player:
    mark: Mark
    model: Model

In [192]:
class Game:
    def __init__(self, player1, player2, m=None, n=None, k=5, view=None, play=True):
        self.width = m
        self.height = n
        self.row = k # row to win
        
        width = m
        height = n
        auto_first_turn = False
        if self.width is None and self.height is None:
            # infinte board
            # let first turn will be on center of the board
            # and accessible field will be doubled row
            height = self.row * 2 + 1
            width = self.row * 2 + 1
            auto_first_turn = True
        elif self.width is not None:
            pass
        elif self.height is not None:
            pass
            
        self.board = np.full((height, width), Mark.NO.value)
        
        self.player1 = Player(Mark.X, player1)
        self.player2 = Player(Mark.O, player2)
        
        if auto_first_turn:
            self.board[self.row][self.row] = self.player1.mark.value
            self.current_player = self.player2
        else:
            self.current_player = self.player1
        
        self.view = view
        if self.view is not None:
            self.view.board = self.board
            self.view.next_player = self.current_player.mark
            self.view.turn_callback = self.apply_action
            self.view.update()
            
        if play:
            self.start()
            
    def apply_action(self, row, column):
        if self.board[row][column] != Mark.NO.value:
            raise RuntimeError('Cell already used')
        self.board[row][column] = self.current_player.mark.value
        
        # check for win
        winner = self.check_win()
        
        # extend board
        if self.width is None or self.height is None:
            self.extend_board()
        
        if winner != Mark.NO.value:
            # obviously, the winner is current player
            self.winner = self.current_player
            self.end_game = True
        else:
            if self.current_player == self.player1:
                self.current_player = self.player2
            elif self.current_player == self.player2:
                self.current_player = self.player1
            else:
                raise RuntimeError('Unknown player')

        self.update_view()
        
        if not self.end_game:
            self.get_action()
            
        
    def get_action(self):
        action = self.current_player.model.get_action()
        if action is None:
            return # wait callback from UI
        else:
            row, column = action
            self.apply_action(row, column)

    def start(self):
        self.winner = None
        self.draw = False
        self.end_game = False
        
        self.get_action()

    def update_view(self):
        if self.view is not None:
            self.view.board = self.board
            if self.winner is not None:
                self.view.winner = self.current_player.mark
            elif self.draw:
                self.view.draw = True
            else:
                self.view.next_player = self.current_player.mark

            self.view.update()
            
    def check_win(self):
        h, w = self.board.shape
        
        # check rows
        for r in range(h):
            player = Mark.NO.value
            count = 0
            for c in range(w):
                player, count = self.check_cell(self.board[r, c], player, count)
                if count == self.row:
                    return player
                
                    
        # check columns
        for c in range(w):
            player = Mark.NO.value
            count = 0
            for r in range(h):
                player, count = self.check_cell(self.board[r, c], player, count)
                if count == self.row:
                    return player
        
        #check diagonal left to right
        #top of the board
        re = h
        for cs in range(w - (self.row - 1)):
            c = cs
            player = Mark.NO.value
            count = 0
            for r in range(re):
                player, count = self.check_cell(self.board[r, c], player, count)
                if count == self.row:
                    return player
                c += 1
            re -= 1
        
        # bottom of the board
        ce = w - 1
        for rs in range(1, h - (self.row - 1)):
            r = rs
            player = Mark.NO.value
            count = 0
            for c in range(ce):
                player, count = self.check_cell(self.board[r, c], player, count)
                if count == self.row:
                    return player
                r += 1
            ce -= 1
        
        #check diagonal right to left
        #top of the board
        re = self.row - 1
        for cs in range(self.row - 1, w):
            c = cs
            player = Mark.NO.value
            count = 0
            for r in range(re + 1): # [) interval is awesome
                player, count = self.check_cell(self.board[r, c], player, count)
                if count == self.row:
                    return player
                c -= 1
            re += 1
        
        # bottom of the board
        for rs in range(1, h - self.row + 1):
            r = rs
            player = Mark.NO.value
            count = 0
            c = w - 1
            while True:
                player, count = self.check_cell(self.board[r, c], player, count)
                if count == self.row:
                    return player
                c -= 1
                r += 1
                if c < 0 or r == h:
                    break
        return Mark.NO.value

    def check_cell(self, cell, player, count):
        if cell == Mark.NO.value:
            player = Mark.NO.value
            count = 0
        elif cell == player:
            count += 1
        else:
            player = cell
            count = 1

        return (player, count)

    def extend_board(self):
        top_rows = 0
        for i, r in enumerate(self.board):
            if r.sum() > 0:
                top_rows = i
                break
                
        bottom_rows = 0
        for i, r in enumerate(reversed(self.board)):
            if r.sum() > 0:
                bottom_rows = i
                break
                
        left_cols = 0
        for i, c in enumerate(self.board.T):
            if c.sum() > 0:
                left_cols = i
                break
                
        right_cols = 0
        for i, c in enumerate(reversed(self.board.T)):
            if c.sum() > 0:
                right_cols = i
                break
                
        add_top = self.row - top_rows
        add_bottom = self.row - bottom_rows
        add_left = self.row - left_cols
        add_right = self.row - right_cols
        
        top = np.zeros((add_top, self.board.shape[1]))
        bottom = np.zeros((add_bottom, self.board.shape[1]))
        
        left = np.zeros((self.board.shape[0] + add_top + add_bottom, add_left))
        right = np.zeros((self.board.shape[0] + add_top + add_bottom, add_right))
        
        self.board = np.c_[left, np.r_[top, self.board, bottom], right]

In [201]:
g = Game(player1=JupyterHumanModel(), player2=JupyterHumanModel(), view=JupyterView())
g.view.out

Output()

TODO:
* UI переделать на jp_proxy_widget